<a href="https://colab.research.google.com/github/omkar806/Final_year_project/blob/main/Music_Mood_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate
import numpy as np
import joblib

In [2]:
features_for_mood = ['energy', 'liveness', 'tempo', 'speechiness',
                                     'acousticness', 'instrumentalness', 'danceability', 'duration_ms',
                                     'loudness', 'valence']

In [3]:
data = pd.read_csv('happy_sad.csv')
hyper_opt = False
data

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,song_emotion,duration_ms
0,0.700,0.526,-6.246,0.0355,0.0981,0.471000,0.1020,0.774,92.015,happy,261559.0
1,0.802,0.763,-6.456,0.0336,0.2200,0.000396,0.1280,0.789,120.078,happy,260507.0
2,0.953,0.584,-7.570,0.0518,0.5430,0.000000,0.0786,0.861,104.007,happy,216999.0
3,0.792,0.630,-6.741,0.0682,0.1020,0.000000,0.0758,0.659,100.022,happy,328052.0
4,0.791,0.558,-7.155,0.0661,0.2760,0.000000,0.0859,0.872,95.077,happy,218836.0
...,...,...,...,...,...,...,...,...,...,...,...
1956,0.665,0.539,-7.213,0.0474,0.8250,0.000006,0.1210,0.368,110.101,sad,210000.0
1957,0.690,0.738,-4.046,0.0533,0.1890,0.000000,0.1090,0.642,113.824,sad,233684.0
1958,0.523,0.788,-5.112,0.0295,0.2380,0.000299,0.1110,0.326,87.972,sad,302250.0
1959,0.697,0.660,-5.459,0.0352,0.4720,0.000000,0.2430,0.301,93.011,sad,247375.0


In [4]:
#split into trainval and test
trainx, testx, trainy, testy = train_test_split(data[features_for_mood], data['song_emotion'], test_size = 0.33,
                                                random_state = 42, stratify=data['song_emotion'])

scaler = StandardScaler()
train_scaled = scaler.fit_transform(trainx)

nn = MLPClassifier(max_iter = 15000, alpha=1.0, hidden_layer_sizes=8)
scores = cross_val_score(nn, train_scaled, trainy, cv=5)
print ("cv score: " + str(scores.mean()))

cv score: 0.7349171334861986


In [5]:
if hyper_opt:
    params = {"alpha": np.logspace(-4, 2, 7), 'hidden_layer_sizes': [1, 2, 5, 10, 20, 50, 100]}
    clf = GridSearchCV(nn, params)
    clf.fit(train_scaled, trainy)
    print("hyperparam optimized score : " + str(clf.best_score_))
    import pdb
    pdb.set_trace()

results = cross_validate(nn, train_scaled, trainy, return_train_score=True)
nn = MLPClassifier(hidden_layer_sizes=8, max_iter=15000, alpha=1.0)

nn.fit(train_scaled, trainy)
test_preds = nn.predict(scaler.transform(testx))
print("Model Accuracy : ")
print(accuracy_score(test_preds, testy))

Model Accuracy : 
0.7083333333333334


In [6]:
# Save the trained model
joblib.dump(nn, 'mlp_model.pkl')

['mlp_model.pkl']

In [7]:
loaded_model = joblib.load('mlp_model.pkl')

In [8]:
new_data = pd.read_csv('test_filtered.csv')

In [9]:
# Assuming 'features_for_mood' contains the list of features used during training
features_for_mood = ['energy', 'liveness', 'tempo', 'speechiness',
                     'acousticness', 'instrumentalness', 'danceability', 'duration_ms',
                     'loudness', 'valence']

In [10]:
# Extract features from the new data
new_data_features = new_data[features_for_mood]

In [11]:
# Load the scaler used during training
scaler = StandardScaler()

In [12]:
# Transform the new data using the scaler
new_data_scaled = scaler.fit_transform(new_data_features)

In [13]:
# Make predictions using the loaded model
predictions = loaded_model.predict(new_data_scaled)

In [14]:
# Add the predictions as a new column to the DataFrame
new_data['predicted_mood'] = predictions

In [15]:
# Save the DataFrame with predictions back to a CSV file
new_data.to_csv('new_data_with_predictions.csv', index=False)

In [16]:
pred_df = pd.read_csv('new_data_with_predictions.csv')

In [17]:
pred_df.head(50)

,name,popularity,date_added,release_year,artists,danceability,energy,key,loudness,mode,...,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,song_emotion,predicted_mood
0,Tumse Hi Tumse,56,2019-04-10 04:35:31+00:00,2010,"['Shekhar Ravjiani', 'Caralisa Monteiro']",0.700,0.526,0,-6.246,1,...,92.015,audio_features,6qNyL8lZuqFmy9ginJRQOZ,spotify:track:6qNyL8lZuqFmy9ginJRQOZ,https://api.spotify.com/v1/tracks/6qNyL8lZuqFm...,https://api.spotify.com/v1/audio-analysis/6qNy...,261559,4,happy,happy
1,Slow Motion Angreza,58,2019-04-10 04:58:41+00:00,2013,"['Shankar-Ehsaan-Loy', 'Sukhwinder Singh', 'Lo...",0.802,0.763,0,-6.456,1,...,120.078,audio_features,0JX7Es2j556FwtR6YqMKzP,spotify:track:0JX7Es2j556FwtR6YqMKzP,https://api.spotify.com/v1/tracks/0JX7Es2j556F...,https://api.spotify.com/v1/audio-analysis/0JX7...,260507,4,happy,happy
2,Lamberghini,48,2019-04-10 05:00:25+00:00,2018,"['The Doorbeen', 'Ragini']",0.953,0.584,11,-7.570,1,...,104.007,audio_features,45P9g6kUb7itdonjjGjDdS,spotify:track:45P9g6kUb7itdonjjGjDdS,https://api.spotify.com/v1/tracks/45P9g6kUb7it...,https://api.spotify.com/v1/audio-analysis/45P9...,216999,4,happy,happy
3,Matargashti,67,2019-04-10 05:00:51+00:00,2015,['Mohit Chauhan'],0.792,0.630,0,-6.741,1,...,100.022,audio_features,5GeBgck1MU2tlIkMpsn8uT,spotify:track:5GeBgck1MU2tlIkMpsn8uT,https://api.spotify.com/v1/tracks/5GeBgck1MU2t...,https://api.spotify.com/v1/audio-analysis/5GeB...,328052,4,happy,happy
4,Kabhi Kabhi Aditi,72,2019-04-10 05:01:00+00:00,2008,['Rashid Ali'],0.791,0.558,7,-7.155,0,...,95.077,audio_features,3APdIdF8H0jsxSuGOqXedS,spotify:track:3APdIdF8H0jsxSuGOqXedS,https://api.spotify.com/v1/tracks/3APdIdF8H0js...,https://api.spotify.com/v1/audio-analysis/3APd...,218836,4,happy,happy
5,"Milegi Milegi (From ""Stree"")",55,2019-04-10 05:01:16+00:00,2018,"['Mika Singh', 'Sachin-Jigar']",0.843,0.892,2,-4.371,1,...,125.991,audio_features,52uiJsA4FrtAZlgqxCJwJI,spotify:track:52uiJsA4FrtAZlgqxCJwJI,https://api.spotify.com/v1/tracks/52uiJsA4FrtA...,https://api.spotify.com/v1/audio-analysis/52ui...,153745,4,happy,happy
6,"Tum Hi Ho Bandhu (From ""Cocktail"")",0,2019-04-10 05:02:14+00:00,2012,"['Neeraj Shridhar', 'Kavita Seth', 'Pritam']",0.714,0.892,11,-5.342,0,...,134.997,audio_features,2zZTXtLqCsICzEcztKjn47,spotify:track:2zZTXtLqCsICzEcztKjn47,https://api.spotify.com/v1/tracks/2zZTXtLqCsIC...,https://api.spotify.com/v1/audio-analysis/2zZT...,282012,4,happy,happy
7,Uff Teri Adaa,67,2019-04-10 05:02:27+00:00,2010,"['Shankar Mahadevan', 'Alyssa Mendonsa']",0.647,0.792,2,-9.815,1,...,132.032,audio_features,0og9wKFGgFFNQnrBe7eisG,spotify:track:0og9wKFGgFFNQnrBe7eisG,https://api.spotify.com/v1/tracks/0og9wKFGgFFN...,https://api.spotify.com/v1/audio-analysis/0og9...,304089,4,happy,happy
8,Ilahi (Reprise),54,2019-04-10 05:02:40+00:00,2013,"['Pritam', 'Mohit Chauhan']",0.568,0.921,9,-4.558,1,...,131.903,audio_features,5wdkKS3kWXBU9bklLVsAFs,spotify:track:5wdkKS3kWXBU9bklLVsAFs,https://api.spotify.com/v1/tracks/5wdkKS3kWXBU...,https://api.spotify.com/v1/audio-analysis/5wdk...,214796,4,happy,sad
9,"Tu Meri (From ""Bang Bang"")",0,2019-04-10 05:04:09+00:00,2014,"['Vishal-Shekhar', 'Vishal Dadlani']",0.679,0.688,8,-5.522,1,...,127.951,audio_features,29GEhH79MHH2wzB7FXRQ2R,spotify:track:29GEhH79MHH2wzB7FXRQ2R,https://api.spotify.com/v1/tracks/29GEhH79MHH2...,https://api.spotify.com/v1/audio-analysis/29GE...,257089,4,happy,happy
